# Data Pre-Processing

## Imports

In [205]:
import pandas as pd
import numpy as np

## Dataset Import

In [206]:
# Ignoring the ID Space mapping to the streaming services because we are not using it
# df_track_id_space = pd.read_csv('download/track_id_space.csv', sep=';')
# df_album_id_space = pd.read_csv('download/album_id_space.csv', sep=';')
# df_artist_id_space = pd.read_csv('download/artist_id_space.csv', sep=';')

# Tracks
df_track = pd.read_csv('download/track.csv', sep=';')
df_album_track = pd.read_csv('download/album_track.csv', sep=';')
df_artist_track = pd.read_csv('download/artist_track.csv', sep=';')
df_track_sim = pd.read_csv('download/track_similarity.csv', sep=';')

# Albums and Artists
df_album = pd.read_csv('download/album.csv', sep=';')
df_artist_album = pd.read_csv('download/artist_album.csv', sep=';')
df_artist = pd.read_csv('download/artist.csv', sep=';')

# Ratings
df_lastfm_rating = pd.read_csv('download/lastfm_rating.csv', sep=';')
df_billboard_rating = pd.read_csv('download/billboard_rating.csv', sep=';')
df_spotify_rating = pd.read_csv('download/spotify_rating.csv', sep=';')

## Track

In [207]:
print(df_track.shape)
print(df_album_track.shape)

# Add information about the album to the track
df_track = pd.merge(df_track, df_album_track, on='track_id', how='left')

df_track = df_track.drop(columns=['position']) # Useless to know the position of the track in the album

print(df_track.shape)

df_track.head()

(23385, 5)
(19486, 3)
(23385, 6)


,track_id,title,release_date,lastfm_playcount,lastfm_listeners,album_id
0,1,Uptown Funk!,2014-11-10 00:00:00,0,0,4.0
1,2,Blank Space,2014-10-27 00:00:00,1822265,92902,1.0
2,3,Take Me To Church,2014-05-20 00:00:00,0,0,1827.0
3,4,Thinking Out Loud,2014-06-21 00:00:00,1654222,233802,2.0
4,5,I'm Not The Only One,2014-07-15 00:00:00,1403471,201114,3.0


In [208]:
print(df_track.shape)
print(df_album.shape)

# add to track the column artist_id
df_track = pd.merge(df_track, df_artist_track, on='track_id', how='left')

# replace "title" by "name"
df_track = df_track.rename(columns={'title': 'name'})

print(df_track.shape)

df_track.head()

(23385, 6)
(1843, 4)
(23385, 7)


,track_id,name,release_date,lastfm_playcount,lastfm_listeners,album_id,artist_id
0,1,Uptown Funk!,2014-11-10 00:00:00,0,0,4.0,1
1,2,Blank Space,2014-10-27 00:00:00,1822265,92902,1.0,2
2,3,Take Me To Church,2014-05-20 00:00:00,0,0,1827.0,3
3,4,Thinking Out Loud,2014-06-21 00:00:00,1654222,233802,2.0,4
4,5,I'm Not The Only One,2014-07-15 00:00:00,1403471,201114,3.0,5


In [209]:
# save to file
df_track.to_csv('data/track.csv', index=False, sep=',')

## Album

In [210]:
print(df_album.shape)
print(df_artist_album.shape)

# Add information about the artist to the album
df_album = pd.merge(df_album, df_artist_album, on='album_id')

print(df_album.shape)

df_album.head()

(1843, 4)
(1843, 2)
(1843, 5)


,album_id,name,lastfm_playcount,lastfm_listeners,artist_id
0,1,1989,5153810,56798,2
1,2,x,1128578,25634,4
2,3,In the Lonely Hour,1314365,71085,5
3,4,Jealous,0,1,7
4,6,Shake It Off,520856,107117,2


In [211]:
# save to file
df_album.to_csv('data/album.csv', index=False, sep=',')

## Artist

In [212]:
print(df_artist.shape)

df_artist = df_artist.drop(columns=['image_url'])

df_artist.head()

(2557, 3)


,artist_id,name
0,1,Mark Ronson Featuring Bruno Mars
1,2,Taylor Swift
2,3,Hozier
3,4,Ed Sheeran
4,5,Sam Smith


In [213]:
def get_all_tracks_by_artist(artist_id, only_id=False):
    return df_track[df_track['artist_id'] == artist_id] if not only_id else df_track[df_track['artist_id'] == artist_id]['track_id']

def get_all_albums_by_artist(artist_id, only_id=False):
    return df_album[df_album['artist_id'] == artist_id] if not only_id else df_album[df_album['artist_id'] == artist_id]['album_id']

# Add the new columns "all_tracks" and "all_albums" to the artist as a list of ids
df_artist['all_tracks'] = df_artist['artist_id'].apply(lambda x: get_all_tracks_by_artist(x, True).tolist())
df_artist['all_albums'] = df_artist['artist_id'].apply(lambda x: get_all_albums_by_artist(x, True).tolist())

df_artist.head()

,artist_id,name,all_tracks,all_albums
0,1,Mark Ronson Featuring Bruno Mars,[1],[]
1,2,Taylor Swift,"[2, 9, 72, 113, 187, 189, 194, 537, 581, 714, ...","[1, 6, 252, 266, 278, 305, 554, 777, 792]"
2,3,Hozier,"[3, 8990, 9009, 9075, 9108, 23197, 23198, 2319...",[1827]
3,4,Ed Sheeran,"[4, 27, 135, 263, 317, 362, 716, 863, 8442, 84...","[2, 210]"
4,5,Sam Smith,"[5, 21, 139, 353, 8373, 8975, 9002, 9094, 9205...",[3]


In [214]:
# save to file
df_artist.to_csv('data/artist.csv', index=False, sep=',')

## Similarity

In [215]:
# remove from sim the column source because it does not have information
df_track_sim = df_track_sim.drop(columns=['source', 'id'])

# rename track_id to track_id_1 and similar_track_id to track_id_2
df_track_sim.rename(columns={'track_id': 'track_id_1', 'similar_track_id': 'track_id_2'}, inplace=True)

df_track_sim.head()

,track_id_1,track_id_2,sim_degree
0,2,72,1.000000
1,2,13,0.441825
2,2,69,0.334081
3,2,3,0.332400
4,2,39,0.309246


In [216]:
# save to file
df_track_sim.to_csv('data/track_similarity.csv', index=False, sep=',')

## Rating

In [217]:
print(df_lastfm_rating.shape)
df_lastfm_rating = df_lastfm_rating.drop(columns=['id'])
df_lastfm_rating['date_lastfm'] = df_lastfm_rating['date_lastfm'].str.replace(' 00:00:00', '')
df_lastfm_rating['date_lastfm'] = pd.to_datetime(df_lastfm_rating['date_lastfm'])

df_lastfm_rating.head()

(43300, 5)


,date_lastfm,position_lastfm,no_of_listeners_lastfm,track_id
0,2006-09-17,1,4220,4574
1,2006-09-17,2,3641,3561
2,2006-09-17,3,3588,4575
3,2006-09-17,4,3413,3875
4,2006-09-17,5,3306,3424


In [218]:
print(df_billboard_rating.shape)
df_billboard_rating = df_billboard_rating.drop(columns=['id'])
df_billboard_rating['date_billboard'] = df_billboard_rating['date_billboard'].str.replace(' 00:00:00', '')
df_billboard_rating['date_billboard'] = pd.to_datetime(df_billboard_rating['date_billboard'])

df_billboard_rating.head()

(57800, 4)


,date_billboard,position_billboard,track_id
0,2015-01-24,1,1
1,2015-01-24,2,2
2,2015-01-24,3,3
3,2015-01-24,4,4
4,2015-01-24,5,5


In [219]:
print(df_spotify_rating.shape)
df_spotify_rating = df_spotify_rating.drop(columns=['id'])
df_spotify_rating['date_spotify'] = df_spotify_rating['date_spotify'].str.replace(' 00:00:00', '')
df_spotify_rating['date_spotify'] = pd.to_datetime(df_spotify_rating['date_spotify'])

df_spotify_rating.head()

(6500, 5)


,date_spotify,position_spotify,no_of_listeners_spotify,track_id
0,2015-01-18,1,4842359,8870
1,2015-01-18,2,3881860,4
2,2015-01-18,3,3840814,3
3,2015-01-18,4,3090497,5
4,2015-01-18,5,2452188,8913


In [220]:
# Collect all unique dates
start_date = pd.to_datetime("2001-01-01")
end_date = pd.to_datetime("2020-01-01")
all_dates = pd.date_range(start=start_date, end=end_date)

print('Number of unique dates:', len(all_dates))
print('First date:', min(all_dates))
print('Last date:', max(all_dates))

# for each date, add a row for each track__id
ids = df_track['track_id'].unique() # unique track ids
dates = all_dates # unique dates
print('Number of unique track ids:', len(ids))

# Create a unified date DataFrame for all dates and all track ids combinations
date_df = pd.DataFrame()
date_df['track_id'] = np.repeat(ids, len(dates))
date_df['date'] = np.tile(dates, len(ids))

date_df['date'] = pd.to_datetime(date_df['date'])
print(date_df.shape)
date_df.head()


Number of unique dates: 6940
First date: 2001-01-01 00:00:00
Last date: 2020-01-01 00:00:00
Number of unique track ids: 23385
(162291900, 2)


,track_id,date
0,1,2001-01-01
1,1,2001-01-02
2,1,2001-01-03
3,1,2001-01-04
4,1,2001-01-05


In [221]:
def align_service_data(service_df, date_df, date_col):
    service_df[date_col] = pd.to_datetime(service_df[date_col])
    service_df = service_df.dropna(subset=[date_col])
    service_df = service_df.rename(columns={date_col: 'date'})
    service_df = service_df.sort_values(by='date').reset_index(drop=True)
    return pd.merge(date_df, service_df, on=['date', 'track_id'], how='left')

date_df = align_service_data(df_lastfm_rating, date_df, 'date_lastfm')
date_df = align_service_data(df_spotify_rating, date_df, 'date_spotify')
date_df = align_service_data(df_billboard_rating, date_df, 'date_billboard')
print(date_df.shape)

# if a row has nan in all 3 position_lastfm, position_spotify and position_billboard, then drop it
date_df = date_df.dropna(subset=['position_lastfm', 'position_spotify', 'position_billboard'], how='all')
print(date_df.shape)

date_df.head()

(162292102, 7)
(105511, 7)


,track_id,date,position_lastfm,no_of_listeners_lastfm,position_spotify,no_of_listeners_spotify,position_billboard
5080,1,2014-11-29,NaN,NaN,NaN,NaN,65.0
5087,1,2014-12-06,NaN,NaN,NaN,NaN,18.0
5094,1,2014-12-13,NaN,NaN,NaN,NaN,8.0
5101,1,2014-12-20,NaN,NaN,NaN,NaN,5.0
5108,1,2014-12-27,NaN,NaN,NaN,NaN,3.0


In [222]:
# save to file
date_df.to_csv('data/rating.csv', index=False, sep=',')